# McKinney Chapter 4 - Practice (Section 4, Wednesday 11:45 AM)

In [1]:
import numpy as np
%precision 4

'%.4f'

## Practice

### Create a 1-dimensional array named `a1` that counts from 0 to 24 by 1.

In [2]:
a1 = np.arange(25)
a1

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])

### Create a 1-dimentional array named `a2` that counts from 0 to 24 by 3.

In [3]:
a2 = np.arange(0, 25, 3)
a2

array([ 0,  3,  6,  9, 12, 15, 18, 21, 24])

In [4]:
np.arange(0, 26, 3)

array([ 0,  3,  6,  9, 12, 15, 18, 21, 24])

In [5]:
np.arange(0, 27, 3)

array([ 0,  3,  6,  9, 12, 15, 18, 21, 24])

### Create a 1-dimentional array named `a3` that counts from 0 to 100 by multiples of 3 or 5.

In [6]:
a3 = np.arange(101)
a3 = a3[(a3 % 3 == 0) | (a3 % 5 == 0)]
a3

array([  0,   3,   5,   6,   9,  10,  12,  15,  18,  20,  21,  24,  25,
        27,  30,  33,  35,  36,  39,  40,  42,  45,  48,  50,  51,  54,
        55,  57,  60,  63,  65,  66,  69,  70,  72,  75,  78,  80,  81,
        84,  85,  87,  90,  93,  95,  96,  99, 100])

We can also create `a3` with a list comprehension.

In [7]:
a3_lc = np.array([i for i in range(101) if (i%3==0) or (i%5==0)])
a3_lc

array([  0,   3,   5,   6,   9,  10,  12,  15,  18,  20,  21,  24,  25,
        27,  30,  33,  35,  36,  39,  40,  42,  45,  48,  50,  51,  54,
        55,  57,  60,  63,  65,  66,  69,  70,  72,  75,  78,  80,  81,
        84,  85,  87,  90,  93,  95,  96,  99, 100])

In [8]:
a3 == a3_lc

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [9]:
(a3 == a3_lc).all()

True

In [10]:
a3 is a3_lc

False

### Create a 1-dimensional array `a3` that contains the squares of the even integers through 100,000.

How much faster is the NumPy version than the list comprehension version?

In [11]:
%timeit np.array([i**2 for i in range(100_001) if i%2==0])

12.8 ms ± 21.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%timeit np.array([i**2 for i in range(0, 100_001, 2)])

10 ms ± 89 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%timeit np.arange(0, 100_001, 2, dtype=np.int64)**2

20.4 µs ± 262 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


---

***Note:***
On some platforms, `np.arange(0, 100_001, 2)` returns an array of 32-bit integers.
If we square this array of 32-bit integers, we get the wrong answer because the large values (e.g., $100,000^2$) are too large to represent as 32-bit integers.
Since we know that we need 54-bit integers for this calculation, we should explcitly set either `dtype='int64'` or `dtype=np.int64`.

In [54]:
np.arange(0, 100_001, 2, dtype='int64')**2

array([          0,           4,          16, ...,  9999200016,
        9999600004, 10000000000])

In [55]:
np.arange(0, 100_001, 2, dtype=np.int64)**2

array([          0,           4,          16, ...,  9999200016,
        9999600004, 10000000000])

This [StackOverflow answer](https://stackoverflow.com/a/1970697/334755) is this best explanation I have found of this behavior.

---

### Write a function that mimic Excel's `pv` function.

Here is how we call Excel's `pv` function:
`=PV(rate, nper, pmt, [fv], [type])`
We can use the annuity and lump sum present value formulas.

Present value of an annuity payment `pmt`:
$PV_{pmt} = \frac{pmt}{rate} \times \left(1 - \frac{1}{(1+rate)^{nper}} \right)$

Present value of a lump sum `fv`:
$PV_{fv} = \frac{fv}{(1+rate)^{nper}}$

In [14]:
def pv(rate, nper, pmt=None, fv=None, type='END'):
    if pmt is None:
        pv_pmt = 0
    else:
        pv_pmt = (pmt / rate) * (1 - (1 + rate)**(-1*nper))
    
    if fv is None:
        pv_fv = 0
    else:
        pv_fv = fv / (1 + rate)**nper

    pv = -1 * (pv_pmt + pv_fv)
    
    if type == 'END':
        pv = pv
    elif type == 'BGN':
        pv *= (1 + rate)
    else:
        print('Enter END or BGN for type argument')
        
    return pv

In [15]:
pv(rate=0.1, nper=10, pmt=100, fv=1000, type='END')

-1000.0000

### Write a function that mimic Excel's `fv` function.

In [16]:
def fv(rate, nper, pmt=None, pv=None, type = 'END'):
    if pmt is not None: # calculate PV of pmt, if given
        fv_pmt = (pmt / rate) * ((1 + rate)**nper - 1)
    else:
        fv_pmt = 0

    if fv is not None: # calculate PV of fv, if given
        fv_pv = pv * (1 + rate)**nper
    else:
        fv_fv = 0
    
    if type=='END': # as-is if end of period payments
        return -1 * (fv_pmt + fv_pv)
    elif type=='BGN': # undo one period of discounting if bgn of period payments
        return -1 * (fv_pmt + fv_pv) * (1 + rate)
    else: # otherwise, ask use to specify end or bgn of period payments
        print('Please enter END or BGN for the type argument')

In [17]:
fv(rate=0.1, nper=10, pmt=100, pv=-1000, type='END')

1000.0000

### Replace the negative values in `data` with -1 and positive values with +1.

In [18]:
np.random.seed(42)
data = np.random.randn(7, 7)
data

array([[ 0.4967, -0.1383,  0.6477,  1.523 , -0.2342, -0.2341,  1.5792],
       [ 0.7674, -0.4695,  0.5426, -0.4634, -0.4657,  0.242 , -1.9133],
       [-1.7249, -0.5623, -1.0128,  0.3142, -0.908 , -1.4123,  1.4656],
       [-0.2258,  0.0675, -1.4247, -0.5444,  0.1109, -1.151 ,  0.3757],
       [-0.6006, -0.2917, -0.6017,  1.8523, -0.0135, -1.0577,  0.8225],
       [-1.2208,  0.2089, -1.9597, -1.3282,  0.1969,  0.7385,  0.1714],
       [-0.1156, -0.3011, -1.4785, -0.7198, -0.4606,  1.0571,  0.3436]])

In [19]:
data_c = data.copy()
data_c[data_c < 0] = -1
data_c[data_c > 0] = +1

data_c

array([[ 1., -1.,  1.,  1., -1., -1.,  1.],
       [ 1., -1.,  1., -1., -1.,  1., -1.],
       [-1., -1., -1.,  1., -1., -1.,  1.],
       [-1.,  1., -1., -1.,  1., -1.,  1.],
       [-1., -1., -1.,  1., -1., -1.,  1.],
       [-1.,  1., -1., -1.,  1.,  1.,  1.],
       [-1., -1., -1., -1., -1.,  1.,  1.]])

In [20]:
data

array([[ 0.4967, -0.1383,  0.6477,  1.523 , -0.2342, -0.2341,  1.5792],
       [ 0.7674, -0.4695,  0.5426, -0.4634, -0.4657,  0.242 , -1.9133],
       [-1.7249, -0.5623, -1.0128,  0.3142, -0.908 , -1.4123,  1.4656],
       [-0.2258,  0.0675, -1.4247, -0.5444,  0.1109, -1.151 ,  0.3757],
       [-0.6006, -0.2917, -0.6017,  1.8523, -0.0135, -1.0577,  0.8225],
       [-1.2208,  0.2089, -1.9597, -1.3282,  0.1969,  0.7385,  0.1714],
       [-0.1156, -0.3011, -1.4785, -0.7198, -0.4606,  1.0571,  0.3436]])

In [21]:
np.where(data < 0, -1, np.where(data > 0, +1, data))

array([[ 1., -1.,  1.,  1., -1., -1.,  1.],
       [ 1., -1.,  1., -1., -1.,  1., -1.],
       [-1., -1., -1.,  1., -1., -1.,  1.],
       [-1.,  1., -1., -1.,  1., -1.,  1.],
       [-1., -1., -1.,  1., -1., -1.,  1.],
       [-1.,  1., -1., -1.,  1.,  1.,  1.],
       [-1., -1., -1., -1., -1.,  1.,  1.]])

In [22]:
np.select(
    condlist=[data<0, data>0],
    choicelist=[-1, +1],
    default=0
)

array([[ 1, -1,  1,  1, -1, -1,  1],
       [ 1, -1,  1, -1, -1,  1, -1],
       [-1, -1, -1,  1, -1, -1,  1],
       [-1,  1, -1, -1,  1, -1,  1],
       [-1, -1, -1,  1, -1, -1,  1],
       [-1,  1, -1, -1,  1,  1,  1],
       [-1, -1, -1, -1, -1,  1,  1]])

### Write a function `npmts()` that calculates the number of payments that generate $x\%$ of the present value of a perpetuity.

Your `npmts()` should accept arguments `c1`, `r`, and `g` that represent  $C_1$, $r$, and $g$.
The present value of a growing perpetuity is $PV = \frac{C_1}{r - g}$, and the present value of a growing annuity is $PV = \frac{C_1}{r - g}\left[ 1 - \left( \frac{1 + g}{1 + r} \right)^t \right]$.

We can use the growing annuity and perpetuity formulas to show: $x = \left[ 1 - \left( \frac{1 + g}{1 + r} \right)^t \right]$. 

Then: $1 - x = \left( \frac{1 + g}{1 + r} \right)^t$.

Finally: $t = \frac{\log(1-x)}{\log\left(\frac{1 + g}{1 + r}\right)}$

***We do not need to accept an argument `c1` because $C_1$ cancels out!***

In [23]:
def npmts(x, r, g):
    return np.log(1-x) / np.log((1 + g) / (1 + r))

In [24]:
npmts(0.5, 0.1, 0.05)

14.899977377480532

### Write a function that calculates the internal rate of return given a NumPy array of cash flows.

Here are some data where the $IRR$ is obvious!

In [25]:
c = np.array([-100, 110])
irr = 0.1

We want to replicate the following calculation with NumPy:

In [26]:
c[0]/(1+irr)**0 + c[1]/(1+irr)**1

-1.4210854715202004e-14

The following NumPy code calculates the present value interest factor for each cash flow:

In [27]:
1 / (1 + irr)**np.arange(len(c)) # present value interest factor

array([1.    , 0.9091])

The following NumPy code calculates the present value for each cash flow:

In [28]:
c / (1 + irr)**np.arange(len(c)) # present value of each cash flow

array([-100.,  100.])

We sum these present values of each cash flow to calculate the net present value:

In [29]:
(c / (1 + irr)**np.arange(len(c))).sum()

-1.4210854715202004e-14

The $IRR$ is the discount rate where $NPV=0$.
We can can use the NumPy code above to try different discount rates until $NPV=0$.
The following code is crude, but sufficient for week three of our class and highlights two tools:

1. Using a `while` to try different discount rates until our answer is within some tolerance
1. Using NumPy to perform repetitive calculations without a `for` loop

In [30]:
def irr(c, guess=0, tol=0.0001, inc=0.0001):
    npv = 42
    irr = guess
    while np.abs(npv) > tol:
        irr += inc
        npv = (c / (1 + irr)**np.arange(len(c))).sum()
        
    return irr

In [31]:
c = np.array([-100, 110])

In [32]:
irr(c)

0.1000

### Write a function `returns()` that accepts *NumPy arrays* of prices and dividends and returns a *NumPy array* of returns.

In [33]:
prices = np.array([100, 150, 100, 50, 100, 150, 100, 150])
dividends = np.array([1, 1, 1, 1, 2, 2, 2, 2])

In [34]:
def returns(p, d):
    return (p[1:] - p[:-1] + d[1:]) / p[:-1]

In [35]:
returns(p=prices, d=dividends)

array([ 0.51  , -0.3267, -0.49  ,  1.04  ,  0.52  , -0.32  ,  0.52  ])

### Rewrite the function `returns()` so it returns *NumPy arrays* of returns, capital gains yields, and dividend yields.

In [36]:
def returns(p, d):
    cg = (p[1:] - p[:-1]) / p[:-1]
    dy = d[1:] / p[:-1]
    r = cg + dy
    return {'r': r, 'cg': cg, 'dy': dy}

In [37]:
returns(p=prices, d=dividends)

{'r': array([ 0.51  , -0.3267, -0.49  ,  1.04  ,  0.52  , -0.32  ,  0.52  ]),
 'cg': array([ 0.5   , -0.3333, -0.5   ,  1.    ,  0.5   , -0.3333,  0.5   ]),
 'dy': array([0.01  , 0.0067, 0.01  , 0.04  , 0.02  , 0.0133, 0.02  ])}

### Rescale and shift numbers so that they cover the range [0, 1]

Input: `np.array([18.5, 17.0, 18.0, 19.0, 18.0])` \
Output: `np.array([0.75, 0.0, 0.5, 1.0, 0.5])`

In [38]:
numbers = np.array([18.5, 17.0, 18.0, 19.0, 18.0])

In [39]:
(numbers - numbers.min()) / (numbers.max() - numbers.min())

array([0.75, 0.  , 0.5 , 1.  , 0.5 ])

### Write functions `var()` and `std()` that calculate variance and standard deviation.

NumPy's `.var()` and `.std()` methods return *population* statistics (i.e., denominators of $n$).
The pandas equivalents return *sample* statistics (denominators of $n-1$), which are more appropriate for financial data analysis where we have a sample instead of a population.


Both function should have an argument `sample` that is `True` by default so both functions return sample statistics by default.

Use the output of `returns()` to compare your functions with NumPy's `.var()` and `.std()` methods.

In [40]:
np.random.seed(42)
r = np.random.randn(1_000_000)
r

array([ 0.4967, -0.1383,  0.6477, ..., -0.113 ,  1.4691,  0.4764])

In [41]:
def var(x, ddof=0):
    N = len(x)
    return ((x - x.mean())**2).sum() / (N - ddof)

In [42]:
r.var()

1.0003761089673018

In [43]:
var(r)

1.0003761089673018

In [44]:
np.allclose(r.var(), var(r))

True

In [45]:
np.allclose(r.var(ddof=1), var(r, ddof=1))

True

In [46]:
def std(x, ddof=0):
    return np.sqrt(var(x=x, ddof=ddof))

In [47]:
r.std()

1.000188036804731

In [48]:
std(r)

1.000188036804731

In [49]:
np.allclose(r.std(), std(r))

True

In [50]:
np.allclose(r.std(ddof=1), std(r, ddof=1))

True